Operations Efficiency|Lookups| Insertions
---|---|---
Sorted (Sequential File)| if search on search key: O(log(N)), otherwise O(N)| O(N)
Unsorted (Heap File)|O(N)| O(1)
**B-tree**| O(log(N)) or O(N) | O(log(N))

- thus OLTP chooses B-tree

# Motivation

- queries often select a subset of records in a table
    - **point query**: find records with a specified attribute value
    - **range query**: find records with attribute values in a given range
- in most cases the query returns only a small subset of the records in a table -> we would like to *locate these records efficiently*
- fundamental file structures have fundamental weaknesses
    - **heap file**: insertion is fast but a lookup requires a *scan*
    - **sequential file**: supports binary search (logarithmic time) but only on *one search key*, all other queries require a *scan*, and furthermore the use of *overflow blocks* degrades performance
- when a file is stored on a disk, even "efficient" binary search can be slow because each record fetched may require a random I/O




## Index (Indices)
- a data structure that speeds up lookup of records by specific search keys
- indices boost performance in two ways
    1. organize data in ways that benefit specific types of queries
    2. usually smaller than the data files they refer to 
        - the buffer pool manager is more likely to hold them in main memory, which avoids expensive I/O operations
        - even when they reside on disk, they can be searched using fewer I/O operations

## Desigining the Physical Schema

- suppose that the logical schema is decided
- first choose a structure for each relation such as sequential, heap, or index-organized
- then add indices with caution
    1. identify *costly* queries that would benefit from the index
        - e.g., queries with where clauses, or queries with joins
    2. choose index type: B+Tree or hash
- adding an index can make **insertion and updates slower**
- some indices may be created automatically by the DBMS, such as on primary and foreign keys


## Basic Concepts

- **Search key**: an attribute or set of attributes used to look up records
- an **index** is a collection of records, called **index entries**, in the form of <search key, pointer>
    - looks a lot like a table, and itself requires some concrete structure for storage

### Types of indices

- **ordered indices**: search keys are stored in sorted order (e.g., variations on a B-tree)
- **hash indices**: search keys are distributed (nearly) uniformly across buckets using a hash function

## Ordered Indices
- **primary index**: the index whose search key coincides with the sequential order of the file (or structure) that stores the table
    - also called **clustered index**
    - search key usually (but not necessarily) equal to the primary key
- **secondary index**: an index whose search key specifies an *order different from the sequential order of the file (or structure)* that stores the table
    - also called **non-clustered index**
- **index-sequential file**: ordered sequential file + a primary index

## Dense Index Files

- **Dense index**: one index record for **every** value of the search key in the file
- both of the examples below are primary indices

<img src="img/Snip20191101_100.png" width=80%/>
<img src="img/Snip20191101_101.png" width=80%/>

## Sparse Index Files

- **Sparse index**: contains index records for **only some** of the values of the search key in the file, possible only when the file is ordered by the same search key
- to locate a record with search key value *K* we
    - inf the index record with largest search key value < *K*
    - scan the file forward starting at the record to which the index record points, stop when searh key value becomes > *K*
- compared to dense indices
    - use less space and incur less maintenance overhead for insertions and deletions
    - generally slower for locating individual records
- tradeoff: sparse index with an index entry for every block in file, corresponding to the lowest search key value in the block

<img src="img/Snip20191101_102.png" width=30%/>

- below example is still primary index

<img src="img/Snip20191101_103.png" width=80%/>

# Secondary Indices

<img src="img/Snip20191101_104.png" width=80%/>

- index record comprises the search key and a set of pointers to matching file records, the pointers may be record numbers or primary key values (requiring a primary index lookup)
- secondary indices are always **dense**

# Primary vs Secondary Indices

- indices enable faster searches
- updating indices incurs overhead: when a table is modified, every index on the table must be updated
- a sequential scan using primary/clustered index is efficient as long as the sequential file has few overflow blocks, but a sequential scan using a secondary index is expensive 
    - each record access may fetch a new block from disk
    - a block fetch from hard disk requires about 5~10 ms if it leads to random I/O, vs ~100ns from memory

## The Need for Advanced Data Structures
- sequential files can be used to store both indices and data
- sequential files enable efficient search, but suffer from the **degradation problem**
    - searching becomes slower as file grows due to overflow blocks
    - the file must be reorganized periodically
- B+Trees: good all around and support efficient range queries
- hashes: good for point queries only

# B+Tree Index Files

- Disadvantage of indexed-sequential files
    - **degradation problem**: searching becomes slower as file grows due to overflow blocks and the file must be reorganized periodically
- Advanatage of B+Tree index files
    - automatically reorganizes itself with *small, local, changes* in the face of insertions and deletions
    - reorganization of entire structure at once is not required to maintain performance
- Disadvantage of B+Trees
    - computation and space overheads

- A B+Tree satisfies many of the useful properties of B-Tree
    - all paths from root to leaf have **equal length**
    - large branching factor, hence **shallow height**
    - each node that is not a root is at least **half full**
- a B+Tree differs from B-Tree in
    - internal nodes only store **separator keys** and pointers to other nodes
    - all the data values are stored in leaf nodes, which are chained together using **sibling pointers**
    - the data values are pointers to records in a file, which can be represented as record numbers or as values of the primary key

- inter-node connections are pointers, nodes that are close together logically (e.g., siblings) are not always close together in terms of their physical storage on disk
    - tree traverslas **may require random I/Os** (tree nodes should align with disk blocks)
    - an in-order walk through the leaf nodes of a B+tree can be slower than scanning a sequential file

- B+trees are shallow
    - with $K$ key-data pairs and $n$ child pointers per internal node, $\textrm{height}<=\lceil{\log_{\lceil n/2\rceil}(K)}\rceil$
    - insertions and deletions run in time proportional to height
    - internal levels easily buffered in main memory, in which case most B+tree operations require only one random I/O

<img src="img/Snip20191104_113.png" width=100%/>

# Indexing Strings with B+trees

- keys are often variable length strings (e.g., VARCHAR in SQL)
    - possible to use a variable branching factor
    - decision to split determined by space utilization, not the number of keys
- **Prefix Compression**: a technique to reduces space overhead
    - keys at internal nodes can be prefixes of full key
        - retain enough characters to distinguish entries in the sub-trees separated by the key value
    - keys can also be compressed by sharing a common prefix

# Index-Organized Tables

- in an index-organized table, the index stores the rows of the table rather than pointers to the rows
- advantage: avoids having to traverse a pointer from the index to the file when performing a row lookup by primary key
- disadvantage: a full traversal of the index structure is required to scan the entire table, which may incur more random I/Os than scanning a sequential file

- InnoDB tables are index-organized tables stored in clustered B+tree indices, if you do not declare a primary key then InnoDB will create one and build a clustered index
- InnoDB also provides in-memory adaptive hash indexing on top of the B-tree indices to speed up lookups on frequently accessed table rows


# Multiple-Key Access

- multiple indices may be useful for certain types of queries


- e.g., `SELECT ID FROM instructor WHERE dept_name = "Finance" AND salary = 80000`
- possible strategies for processing query using indices on single attributes
    1. use index on dept_name to find instructors with dept_name Finance; test salary = 80000
    2. use index on salary and test dept_name
    3. use dept_name to find pointers, use salary to find pointers; take intersection of both sets of pointers


## Indices on Multiple Attributes

- **Composite search keys**: search keys containing more than one attribute
    - e.g., (dept_name, salary)
- Composite search keys are compared using **lexicographic ordering**
    - $(a_1, a_2) < (b_1, b_2)$ if 
        - $a_1 < b_1$, or
        - $a_1 = b_1$, and $a_2 < b_2$

- suppose we have an index on the composite search key (dept_name, saslary)
    - efficiently handle queries like
        - `WHERE dept_name = "Finance" AND salary = 80000`
        - `WHERE dept_name = "Finance" AND salary < 80000`
    - but inefficient for
        - `WHERE dept_name < "Finance" AND salary = 80000`
    - the index can be used to find records that satisfy the first condition but do not always satisfy the second condition
